In [26]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot

En primera base se probó con la señal del par EUR-USD para ver esta como se comportaba, ahora probaremos con varias señales de entrada, para ver como se comporta la señal EUR-USD.

Para empezar se probará con los pares EUR/USD y GBP/USD para predecir la señal EUR/USD

In [27]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("../data/TrueFX/GBP-USD/GBPUSD-preprocesado-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [28]:
data_GBPvsUSD.head()

,bid,ask
date,,
2018-01-02 00:00:40,1.34979,1.35176
2018-01-02 00:01:00,1.34918,1.35208
2018-01-02 00:01:10,1.34915,1.35354
2018-01-02 00:01:20,1.34915,1.35386
2018-01-02 00:01:30,1.34938,1.35440


In [29]:
data_EURvsUSD.head()

,bid,ask
date,,
2018-01-01 23:58:00,1.20093,1.20115
2018-01-01 23:58:10,1.20093,1.20111
2018-01-01 23:58:20,1.20082,1.20111
2018-01-01 23:58:30,1.20043,1.20110
2018-01-01 23:58:50,1.20011,1.20113


In [30]:
print('tamaño de la señal EUR/USD', data_EURvsUSD.shape)
print('tamaño de la señal GBP/USD', data_GBPvsUSD.shape)

tamaño de la señal EUR/USD (183535, 2)
tamaño de la señal GBP/USD (185443, 2)


como se puede ver, las dos señales no tienen el mismo tamaño, como la señal predominante es el par GBP/USD, el indice de este será el tenido en cuenta para la construcción de la multiseñal.

In [31]:
%run ../code/EUtilities.py
EU = EUtilities()

In [32]:
#se construye el dataset GBP/USD con las dos señales en el, para su posterior uso
data_GBPvsUSD = EU.multi_signal(data_GBPvsUSD,data_EURvsUSD)

In [34]:
data_GBPvsUSD[:10]

,bid,ask,bidnew
date,,,
2018-01-02 00:00:40,1.34979,1.35176,1.20015
2018-01-02 00:01:00,1.34918,1.35208,1.20030
2018-01-02 00:01:10,1.34915,1.35354,1.34915
2018-01-02 00:01:20,1.34915,1.35386,1.20030
2018-01-02 00:01:30,1.34938,1.35440,1.34938
2018-01-02 00:01:40,1.34938,1.35386,1.34938
2018-01-02 00:01:50,1.34942,1.35300,1.34942
2018-01-02 00:02:10,1.34939,1.35300,1.34939
2018-01-02 00:02:30,1.34979,1.35154,1.34979


ya con la multiseñal se procede a crear el X...

In [35]:
#los datos del par EUR/USD 
X_EURUSD ,_= EU.build_dataset(data_GBPvsUSD.iloc[:,2],window=3,binary_target=False,PNL=False
                                             ,delete_constant_values=True)
#los datos del par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD, y_GBPUSD, bt_GBPUSD = EU.build_dataset(data_GBPvsUSD.iloc[:,0],window=3,binary_target=True
                                                 ,delete_constant_values=True,PNL=True)

In [36]:
X_EURUSD.shape, X_GBPUSD.shape

((160808, 3), (160804, 4))

debido a que son diferentes señales, al habilitar la opción de <i>"delete_constant_values"</i> en la función <i>"build_dataset"</i> esto hace que las señales resultantes no sean iguales en tamaño, por ende para entrenar se cogeran los valores de los indices de la señal a medir, si este no está en la otra señal, se coloca lo de la señal GBP/USD en ese instante de tiempo

In [37]:
X_EURUSD = X_EURUSD.loc[X_GBPUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((160804, 3), (160804, 4))

In [38]:
X_EURUSD[:10]

,0,1,2
date,,,
2018-01-02 00:01:40,1.20030,1.34915,1.20030
2018-01-02 00:02:10,1.20030,1.34938,1.34938
2018-01-02 00:02:30,1.34938,1.34938,1.34942
2018-01-02 00:02:40,1.34938,1.34942,1.34939
2018-01-02 00:03:10,1.34939,1.34979,1.34979
2018-01-02 00:03:20,1.34979,1.34979,1.20026
2018-01-02 00:04:10,1.20024,1.34979,1.20011
2018-01-02 00:05:30,1.20047,1.34940,1.34940
2018-01-02 00:05:40,1.34940,1.34940,1.20055


In [39]:
#para cambiar los valores NaN por los que hay en la señal GBP/USD
X_EURUSD = X_EURUSD.fillna(X_GBPUSD,axis=0)

se unen para entrenar la multiseñal...

In [40]:
X_GBPUSD.insert(loc=3,column=3,value=X_EURUSD.iloc[:,0])
X_GBPUSD.insert(loc=4,column=4,value=X_EURUSD.iloc[:,1])
X_GBPUSD.insert(loc=5,column=5,value=X_EURUSD.iloc[:,2])

In [41]:
X_GBPUSD.head()

,0,1,2,3,4,5,PNL
date,,,,,,,
2018-01-02 00:01:40,1.34918,1.34915,1.34915,1.20030,1.34915,1.20030,0.00023
2018-01-02 00:02:10,1.34915,1.34938,1.34938,1.20030,1.34938,1.34938,0.00004
2018-01-02 00:02:30,1.34938,1.34938,1.34942,1.34938,1.34938,1.34942,-0.00003
2018-01-02 00:02:40,1.34938,1.34942,1.34939,1.34938,1.34942,1.34939,0.00040
2018-01-02 00:03:10,1.34939,1.34979,1.34979,1.34939,1.34979,1.34979,-0.00039


se prueba con el GaussianNB

In [42]:
%run ../code/PNLEstimatorWrapper.py
%run ../code/EUtilities.py
EU = EUtilities()

In [43]:
from sklearn.naive_bayes import GaussianNB
    
y = bt_GBPUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_GBPUSD.index.date)[n_bdtrain:]

est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column='PNL')
result_GNB = EU.step_validation(est_GNB, X_GBPUSD, y, EU.v_split(X_GBPUSD,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/GBP-USD/MultiSeñal/GaussianNB/pnl-GBP-USD_EUR-USD_GNB_wsize3_2018-01.csv')

--------- GaussianNB ----------------
average PNL :  -0.00199052631579
average buy:  2261.31578947
average sell:  4741.89473684
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08  0.00680  1225.0  5574.0          41.0         2316.0
2018-01-09  0.00676  3011.0  3822.0        1606.0         2579.0
2018-01-10  0.00762  3437.0  3611.0         849.0          108.0
2018-01-11  0.00036  4790.0  2343.0        3361.0         1274.0
2018-01-12 -0.01519  3032.0  4189.0        2399.0          477.0
2018-01-15  0.00606  7280.0     0.0        7280.0            0.0
2018-01-16  0.00038     0.0  7107.0           0.0         7107.0
2018-01-17 -0.00455     0.0  7515.0           0.0         7515.0
2018-01-18 -0.00646     0.0  7473.0           0.0         7473.0
2018-01-19  0.00683     0.0  7361.0           0.0         7361.0
2018-01-22 -0.01562     0.0  7399.0           0.0         7399.0
2018-01-23 -0.00114     0.0  7330.0           0.0     

se prueba con el KNeighborsClassifier

In [44]:
from sklearn.neighbors import KNeighborsClassifier
    
y = bt_GBPUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_GBPUSD.index.date)[n_bdtrain:]

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column='PNL')
result_KNC = EU.step_validation(est_KNC, X_GBPUSD, y, EU.v_split(X_GBPUSD,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('--------- KNeighborsClassifier ----------------')
print('average PNL : ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/GBP-USD/MultiSeñal/KNeighborsClassifier/pnl-GBP-USD_EUR-USD_KNC_wsize3_2018-01.csv')

--------- KNeighborsClassifier ----------------
average PNL :  0.00288421052632
average buy:  3156.21052632
average sell:  3847.0
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.00228  3354.0  3445.0          63.0           72.0
2018-01-09  0.00424  4795.0  2038.0         138.0           78.0
2018-01-10  0.00926  4599.0  2449.0         256.0           45.0
2018-01-11  0.01506  4249.0  2884.0         291.0           46.0
2018-01-12 -0.01177  1655.0  5566.0          40.0         1284.0
2018-01-15 -0.00926  1287.0  5993.0          22.0          578.0
2018-01-16 -0.00674  2484.0  4623.0          46.0           88.0
2018-01-17  0.01069  2065.0  5450.0          46.0          156.0
2018-01-18  0.00270  4556.0  2917.0          82.0          152.0
2018-01-19  0.00969  3246.0  4115.0          88.0          138.0
2018-01-22 -0.00146  2499.0  4900.0          22.0          620.0
2018-01-23  0.01944  3261.0  4069.0         113.0   

se prueba con el DecisionTreeClassifier

In [45]:
from sklearn.tree import DecisionTreeClassifier
    
y = bt_GBPUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_GBPUSD.index.date)[n_bdtrain:]

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column='PNL')
result_DTC = EU.step_validation(est_DTC, X_GBPUSD, y, EU.v_split(X_GBPUSD,n_bdtrain,n_bdtest,mday))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('--------- DecisionTreeClassifier ----------------')
print('average PNL : ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/GBP-USD/MultiSeñal/DecisionTreeClassifier/pnl-GBP-USD_EUR-USD_DTC_wsize3_2018-01.csv')

--------- DecisionTreeClassifier ----------------
average PNL :  0.00257789473684
average buy:  3171.05263158
average sell:  3832.15789474
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08  0.00528  4415.0  2384.0          73.0           34.0
2018-01-09  0.00094  4340.0  2493.0          85.0           34.0
2018-01-10 -0.00020  4593.0  2455.0          89.0           39.0
2018-01-11  0.00406  3996.0  3137.0         135.0           63.0
2018-01-12 -0.01687  1722.0  5499.0          81.0           91.0
2018-01-15  0.00544  1035.0  6245.0          49.0         3970.0
2018-01-16  0.00672  3414.0  3693.0         129.0           42.0
2018-01-17  0.01605  2584.0  4931.0          24.0           74.0
2018-01-18  0.00072  2879.0  4594.0         100.0           49.0
2018-01-19  0.00711  4078.0  3283.0          81.0           50.0
2018-01-22 -0.01628  2444.0  4955.0          24.0          360.0
2018-01-23  0.00708  3392.0  3938.0        

se prueba con el RandomForestClassifier

In [46]:
from sklearn.ensemble import RandomForestClassifier
    
y = bt_GBPUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_GBPUSD.index.date)[n_bdtrain:]

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column='PNL')
result_RFC = EU.step_validation(est_RFC, X_GBPUSD, y, EU.v_split(X_GBPUSD,n_bdtrain,n_bdtest,mday))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('--------- RandomForestClassifier ----------------')
print('average PNL : ', np.mean(result_RFC[:,0]))
print('average buy: ', np.mean(result_RFC[:,1]))
print('average sell: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/GBP-USD/MultiSeñal/RandomForestClassifier/pnl-GBP-USD_EUR-USD_RFC_wsize3_2018-01.csv')

--------- RandomForestClassifier ----------------
average PNL :  0.00427052631579
average buy:  2984.26315789
average sell:  4018.94736842
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08  0.00456  1915.0  4884.0          34.0          120.0
2018-01-09  0.00078  5210.0  1623.0          74.0           23.0
2018-01-10  0.00442  3968.0  3080.0         273.0           29.0
2018-01-11  0.00782  3801.0  3332.0         301.0           49.0
2018-01-12 -0.01595  2024.0  5197.0          35.0           91.0
2018-01-15  0.00210   963.0  6317.0          43.0         4035.0
2018-01-16  0.00696  2521.0  4586.0          25.0           89.0
2018-01-17  0.01167  2362.0  5153.0          26.0           91.0
2018-01-18  0.00980  3567.0  3906.0          60.0          106.0
2018-01-19  0.01239  3423.0  3938.0          46.0           56.0
2018-01-22 -0.00092  2254.0  5145.0          24.0         1352.0
2018-01-23  0.02626  2963.0  4367.0        